In [6]:

from google.colab import files
 
 
uploaded = files.upload()

Saving total_output.csv to total_output.csv


In [10]:
!pip install pydot --quiet
!pip install gensim==3.8.3 --quiet
!pip install tensorflow-datasets --quiet
!pip install -U tensorflow-text==2.8.2 --quiet
!pip install transformers --quiet

     |████████████████████████████████| 24.2 MB 580 kB/s 
     |████████████████████████████████| 4.9 MB 13.0 MB/s 
     |████████████████████████████████| 497.9 MB 33 kB/s 
     |████████████████████████████████| 462 kB 72.6 MB/s 
     |████████████████████████████████| 1.4 MB 74.4 MB/s 
     |████████████████████████████████| 5.8 MB 54.5 MB/s 
     |████████████████████████████████| 5.5 MB 11.0 MB/s 
     |████████████████████████████████| 7.6 MB 76.1 MB/s 
     |████████████████████████████████| 163 kB 75.4 MB/s 


In [11]:
import pandas as pd
import numpy as np
import sys
import os
import re

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.layers import Embedding, Input, Dense, Lambda
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
import tensorflow_datasets as tfds
import tensorflow_text as tf_text

from transformers import BertTokenizer, TFBertModel


import sklearn as sk

import nltk
from nltk.data import find

import matplotlib.pyplot as plt



#This continues to work with gensim 3.8.3.  It doesn't yet work with 4.x.  
#Make sure your pip install command specifies gensim==3.8.3
import gensim

In [7]:
import pandas as pd

df=pd.read_csv("total_output.csv")

In [8]:
df.head()

,Unnamed: 0,LineId,Date,Time,Type,Admin,Content,EventId,EventTemplate,Label
0,0,1,9/27/2016,4:30:30,Info,CBS,Starting TrustedInstaller initialization.,4351fc0e,<*> TrustedInstaller <*>,0
1,1,2,9/28/2016,4:30:30,Info,CBS,Loaded Servicing Stack v6.1.7601.23505 with Co...,b642288b,Loaded Servicing Stack v<*> with Core: C:\Wind...,0
2,2,3,9/28/2016,4:30:31,Info,CSI,00000001@2016/9/27:20:30:31.455 WcpInitialize ...,a742c417,<*>@<*>/<*>/<*>:<*>:<*>:<*>.<*> WcpInitialize ...,0
3,3,4,9/28/2016,4:30:31,Info,CSI,00000002@2016/9/27:20:30:31.458 WcpInitialize ...,a742c417,<*>@<*>/<*>/<*>:<*>:<*>:<*>.<*> WcpInitialize ...,0
4,4,5,9/28/2016,4:30:31,Info,CSI,00000003@2016/9/27:20:30:31.458 WcpInitialize ...,a742c417,<*>@<*>/<*>/<*>:<*>:<*>:<*>.<*> WcpInitialize ...,0


In [12]:
nltk.download('word2vec_sample')

word2vec_sample = str(find('models/word2vec_sample/pruned.word2vec.txt'))

model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_sample, binary=False)

[nltk_data] Downloading package word2vec_sample to /root/nltk_data...
[nltk_data]   Unzipping models/word2vec_sample.zip.


In [14]:
whole_data_emb = df["Content"].values.tolist()
whole_data_label=df["Label"].values.tolist()

In [15]:
len(whole_data_emb)

79966

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(whole_data_emb, whole_data_label, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2

In [17]:
#@title Embedding Matrix Creation

EMBEDDING_DIM = len(model['university'])      # we know... it's 300

# initialize embedding matrix and word-to-id map:
embedding_matrix = np.zeros((len(model.vocab.keys()) + 1, EMBEDDING_DIM))       
vocab_dict = {}

# build the embedding matrix and the word-to-id map:
for i, word in enumerate(model.vocab.keys()):
    embedding_vector = model[word]
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
        vocab_dict[word] = i

# we can use the last index at the end of the vocab for unknown tokens
vocab_dict['[UNK]'] = len(vocab_dict)

In [18]:
SEQUENCE_LENGTH = 100

In [19]:
def docs_to_vocab_ids(tokenized_texts_list):
    """
    converting a list of strings to a list of lists of word ids
    """
    texts_vocab_ids = []
    text_labels = []
    valid_example_list = []
    for i, token_list in enumerate(tokenized_texts_list):

        # Get the vocab id for each token in this doc ([UNK] if not in vocab)
        vocab_ids = []
        for token in list(token_list.numpy()):
            decoded = token.decode('utf-8', errors='ignore')
            if decoded in vocab_dict:
                vocab_ids.append(vocab_dict[decoded])
            else:
                vocab_ids.append(vocab_dict['[UNK]'])
            
        # Truncate text to max length, add padding up to max length
        vocab_ids = vocab_ids[:SEQUENCE_LENGTH]
        n_padding = (SEQUENCE_LENGTH - len(vocab_ids))
        # For simplicity in this model, we'll just pad with unknown tokens
        vocab_ids += [vocab_dict['[UNK]']] * n_padding
        valid_example_list.append(i)
        # Add this example to the list of converted docs
        texts_vocab_ids.append(vocab_ids)
            
        if i % 5000 == 0:
            print('Examples processed: ', i)

    print('Total examples: ', i)
    return (np.array(texts_vocab_ids), valid_example_list)

In [20]:
tokenizer = tf_text.WhitespaceTokenizer()
train_tokens = tokenizer.tokenize(X_train)
                            

In [21]:
train_valid_example_list = docs_to_vocab_ids(train_tokens)

Examples processed:  0
Examples processed:  5000
Examples processed:  10000
Examples processed:  15000
Examples processed:  20000
Examples processed:  25000
Examples processed:  30000
Examples processed:  35000
Examples processed:  40000
Examples processed:  45000
Examples processed:  50000
Examples processed:  55000
Total examples:  59973


In [22]:
X_train[0:3]

['Applicability(ComponentAnalyzerEvaluateSelfUpdate): Component: amd64_microsoft-windows-b..os-loader.resources_31bf3856ad364e35_6.1.7601.23126_pt-pt_d7bd5bca7feafe17, elevate: 2, applicable(true/false): 0',
 'Appl: Selfupdate, Component: x86_microsoft-windows-m..ditevtlog.resources_31bf3856ad364e35_0.0.0.0_nb-no_38ac3fc129e29535 (6.1.7601.23126), elevation:2, lower version revision holder: 0.0.0.0',
 'Appl: Selfupdate, Component: amd64_microsoft-windows-winsrv_31bf3856ad364e35_0.0.0.0_none_367727d39cdce317 (6.1.7601.23126), elevation:2, lower version revision holder: 6.1.7601.19160']

In [23]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [25]:
num_train_examples = 30000
num_test_examples = 20000

max_length = 100


x_train = bert_tokenizer(X_train[:num_train_examples], 
              max_length=max_length,
              truncation=True,
              padding='max_length', 
              return_tensors='tf')
y_train = y_train[:num_train_examples]



def select_min_length_examples(x_data, y_data):

  x_input_ids = []
  y_labels = []

  for ((input_ids, masks), label) in zip(zip(x_data['input_ids'], x_data['attention_mask']), y_data):
    if masks[-1] == 1:
      x_input_ids.append(input_ids)
      y_labels.append(label)

  return np.array(x_input_ids), np.array(y_labels) 


In [26]:
x_train

{'input_ids': <tf.Tensor: shape=(30000, 100), dtype=int32, numpy=
array([[  101,   138,  8661, ...,  2276,   120,   102],
       [  101,   138,  8661, ...,   102,     0,     0],
       [  101,   138,  8661, ...,     0,     0,     0],
       ...,
       [  101,   138,  8661, ...,     0,     0,     0],
       [  101,   138,  8661, ...,  6014,   114,   102],
       [  101, 16409, 26426, ...,     0,     0,     0]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(30000, 100), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(30000, 100), dtype=int32, numpy=
array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 0, 0, 0]], dtype